In [15]:
"""File to generate sample data for testing."""
from datetime import date, timedelta, datetime
import random
import math
import pandas as pd
import numpy as np
import plotly.graph_objects as go



In [16]:
#Initialization of variables
days = 90
amplitude = 20
displacement = 50
noise = 0.4
day = date.today().day
month = date.today().month
year = date.today().year

if month == 1:
    year = year - 1
    month = 12

last_month_date = date(year, month -1, day)
last_week_date = date.today() - timedelta(days = 7)
last_year_date = date.today() - timedelta(days = 365)

colors = {
    'background': '#111111',
    'text': '#7FDBFF'
}

# Initial Lists
test_type = ['liquid based', 'conventional','another type', 'fourth type', 'just another']
adequacy_list = [{'satisfactory': 'Yes'},{'satisfactory': 'No', 'processed': 'Not processed'},{'satisfactory': 'No', 'processed': 'processed'}]
results_list = ['negative', 'ASC-US', 'ASC-H', 'LSIL', 'MSIL', 'SCC', 'AC', 'JJJ', 'KKK', 'LLL']
genotype_list = ['16','18']
choices = ['true positive', 'false positive', 'false negative', 'true negative']
initial_df = pd.DataFrame()
test_df = pd.DataFrame(columns=['day', 'type', 'adequacy', 'result', 'mvp', 'cytology', 'hystology'])

In [17]:
#Data Generator
def sin_data_generate(N,A,S,R):
    """To generate sinusoidal data."""
    x = np.arange(0,N,1)
    noise = []
    for j in range(N):
        noise.append(R*random.choice(range(-3,3)))
    noise = np.array(noise)
    y = A*np.sin(30*x/(2*math.pi))+S
    y = y + noise
    return [x,y]

In [18]:

for type in test_type:
    test_data = sin_data_generate(days, random.randint(5, 20), random.randint(20,50), noise)
    final = datetime.date(datetime.now())
    initial = final - timedelta(days=days-1)
    # print(initial, final)
    daterange = pd.date_range(initial, final, freq='D')

    initial_df['day'] = daterange
    initial_df[type] = test_data[1].astype(int)
    # test_df['adequacy ' + type] = random.choices(adequacy_list, k=len(test_df))
    # test_df['result ' + type] = random.choices(results_list, k=len(test_df))
    # test_df['mvp'] = random.choices(genotype_list, k=len(test_df))
    # test_df['cytology'] = random.choices(['positive', 'negative'], k=len(test_df), weights=[0.8, 0.2])
    # test_df['hystology'] = random.choices(['positive', 'negative'], k=len(test_df), weights=[0.8, 0.2])

# print(test_df.count)
(initial_df)

,day,liquid based,conventional,another type,fourth type,just another
0,2022-09-08,20,38,43,27,30
1,2022-09-09,10,27,32,17,12
2,2022-09-10,18,38,41,28,27
3,2022-09-11,30,49,50,41,47
4,2022-09-12,23,41,44,31,34
...,...,...,...,...,...,...
85,2022-12-02,15,34,38,21,20
86,2022-12-03,28,47,49,37,43
87,2022-12-04,27,44,48,36,41
88,2022-12-05,14,30,36,21,17


In [19]:
print(test_df.columns)
for type in test_type:
    type_list_samples = initial_df[type].values
    for index, number in enumerate(type_list_samples):
        # print(f'number: {number}')
        # print(f'dataframe: {(initial_df[type].values)}')
        for sample in range(number):
            day = initial_df['day'].iloc[index]
            adequacy = random.choices(adequacy_list)[0]
            result = random.choices(results_list)[0]
            mvp = random.choices(genotype_list)[0]
            cytology = random.choices(['positive', 'negative'], weights=[0.8, 0.2])[0]
            hystology = random.choices(['positive', 'negative'], weights=[0.8, 0.2])[0]
            new_row = {'day': day,'type': type, 'adequacy': adequacy, 'result': result, 'mvp': mvp, 'cytology': cytology, 'hystology': hystology}
            # test_df = test_df.apply(lambda row: add_row(row))
            new_df = pd.DataFrame([new_row])
            test_df = pd.concat([test_df, new_df], axis=0, ignore_index=True)
            new_df.reset_index()

            # print(f'sample: {test_df["type"]}')
                # test_df = test_df.concat(new_row, ignore_index=True)
                # test_df['type'] = type

(test_df.tail(5))
    # test_df['test_quality'] = np.select([
    #     (test_df['cytology'] == 'positive') & (test_df['hystology'] == 'positive'),
    #     (test_df['cytology'] == 'positive') & (test_df['hystology'] == 'negative'),
    #     (test_df['cytology'] == 'negative') & (test_df['hystology'] == 'positive'),
    #     (test_df['cytology'] == 'negative') & (test_df['hystology'] == 'negative')
    #     ], choices, default='true negative')


Index(['day', 'type', 'adequacy', 'result', 'mvp', 'cytology', 'hystology'], dtype='object')


,day,type,adequacy,result,mvp,cytology,hystology
14195,2022-12-06,just another,"{'satisfactory': 'No', 'processed': 'Not proce...",LSIL,18,positive,positive
14196,2022-12-06,just another,"{'satisfactory': 'No', 'processed': 'Not proce...",JJJ,18,negative,negative
14197,2022-12-06,just another,"{'satisfactory': 'No', 'processed': 'Not proce...",JJJ,18,positive,positive
14198,2022-12-06,just another,"{'satisfactory': 'No', 'processed': 'Not proce...",AC,16,positive,positive
14199,2022-12-06,just another,{'satisfactory': 'Yes'},LLL,16,positive,positive


In [100]:
panda = test_df.loc[test_df['type'] == 'liquid based']
panda = panda.loc[panda['result'] == 'LLL']
frame = panda.groupby('day')
another_frame = pd.DataFrame()
another_frame['count'] = frame['result'].count()
another_frame = another_frame.reset_index()
# another_frame
graph = go.Figure()
graph.add_trace(go.Scatter(x=another_frame['day'], y= another_frame['count'], mode="markers+lines", name='LLL', marker_symbol= 1, showlegend=True))
graph


In [1]:

graph = go.Figure()
if test_type[0] == 'negative':
    visibility = True
else:
    visibility = 'legendonly'

# graph.add_trace(go.Scatter(x=test_df['day'], y=initial_df[type], mode='lines+markers', name=test_type[0], cliponaxis=True,
# line_shape= "spline", showlegend=True, line_smoothing=0, visible=visibility
#     ))
count = 0
for item in results_list:
    count += 1
    filter1 = test_df['result'] == item
    data = test_df[filter1]
    data.sort_values('day', inplace=True)
    day_group = data.groupby('day')
    # sample_df = test_df[filter1]
    count_data = day_group['result'].value_counts()
    count_data    
    
    # df = data[data['result'] == item]
    # day_grp = df.groupby(['day'])
    # df['count'] = test['result'].count()
    mock = pd.DataFrame()
    mock['day'] = data['day']
    mock.sort_values('day', inplace=True)
    mock['count'] = count_data
    # mock['count'] = mock['count'].fillna(0)
    # mock = test['type'].count()
    # graph.add_trace(go.Scatter(x=mock['day'], y= mock['count'], mode="markers+lines", name=item, marker_symbol= count, showlegend=True))
# test['type'].count()
# (type_list_samples)
# day_grp['result'].value_counts()
mock

NameError: name 'go' is not defined

In [ ]:
# counts = df['day'].value_counts()
counts = df.groupby(['day'])
counts['result'].value_counts().head(50)
# counts['day']= df['day'].unique()
# counts.reset_index()
